In [2]:
%%writefile ../src/OrderEvent.sol
pragma solidity ^0.6.0;

contract OrderEvent {
    uint unitPrice = 10;
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        uint256 orderTime = now;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Writing ../src/OrderEvent.sol


In [4]:
!solc --abi --bin --gas ../src/OrderEvent.sol


======= ../src/OrderEvent.sol:OrderEvent =======
Gas estimation:
construction:
   20159 + 102600 = 122759
external:
   order(bytes2,uint256):	5601
Binary:
6080604052600a60005534801561001557600080fd5b50610201806100256000396000f3fe60806040526004361061001e5760003560e01c806332af7f7314610023575b600080fd5b61007a6004803603604081101561003957600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff191690602001909291908035906020019092919050505061007c565b005b6000429050600080548302905080341461009557600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dffffffffffffffffffffffff

In [11]:
%%writefile ../src/OrderEventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
var _bin="0x" + "6080604052600a60005534801561001557600080fd5b50610201806100256000396000f3fe60806040526004361061001e5760003560e01c806332af7f7314610023575b600080fd5b61007a6004803603604081101561003957600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff191690602001909291908035906020019092919050505061007c565b005b6000429050600080548302905080341461009557600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200191505060405180910390a35050505056fea264697066735822122046c1950317a894b781f0187cac79554ac712ddea1f1cbfcf4eb988afdc6e477864736f6c63430006040033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 300000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting ../src/OrderEventDeploy.js


In [12]:
!node ../src/OrderEventDeploy.js

Deploying the contract from 0x78e90829aA2f4006eB2eab0945b250ebf34329A6
hash: 0xcee8c6ec8345210bde97febe9ffde93420be4fb895c9f7c3fee6bca9b0c3f713
---> The contract deployed to: 0x3A926AFf2f59316B9b2635871694150eB7DB4851


In [70]:
%%writefile ../src/OrderEventUse.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray=JSON.parse(_abiStr);

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x3A926AFf2f59316B9b2635871694150eB7DB4851');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            filter: {_from: accounts[0], _value: 30},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            
            console.log("Event fired: " + JSON.stringify(result.returnValues));
            
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
  
    var my = await _order.methods.order("0x1234", 3)
        .send({from: accounts[0], gas: 100000, value:30})

    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));

    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));

    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting ../src/OrderEventUse.js


In [ ]:
!node ../src/OrderEventUse.js

In [16]:
%%writefile ../src/FallbackTest.sol
pragma solidity ^0.6.0;
contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    fallback () external {
        emit PrintLog("fallback called");
    }
}

Writing ../src/FallbackTest.sol


In [18]:
!solc ../src/FallbackTest.sol --combined-json abi

{"contracts":{"../src/FallbackTest.sol:FallbackTest":{"abi":"[{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"stateMutability\":\"nonpayable\",\"type\":\"fallback\"},{\"inputs\":[],\"name\":\"callA\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"pure\",\"type\":\"function\"}]"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


In [23]:
%%writefile ../src/FallbackTestABI.json
{"contracts":{"../src/FallbackTest.sol:FallbackTest":{"abi":"[{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"stateMutability\":\"nonpayable\",\"type\":\"fallback\"},{\"inputs\":[],\"name\":\"callA\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"pure\",\"type\":\"function\"}]"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Overwriting ../src/FallbackTestABI.json


In [24]:
!solc ../src/FallbackTest.sol --combined-json bin

{"contracts":{"../src/FallbackTest.sol:FallbackTest":{"bin":"608060405234801561001057600080fd5b50610190806100206000396000f3fe608060405234801561001057600080fd5b506004361061002f5760003560e01c8063e7f09e051461009a57610030565b5b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100a261011d565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100e25780820151818401526020810190506100c7565b50505050905090810190601f16801561010f5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040518060400160405280600b81526020017f646f696e672063616c6c4100000000000000000000000000000000000000000081525090509056fea2646970667358221220e82fa1bc4dc7ed37bbfad314b7799e14b2b537797a926ad6a5ca3317667e3f4064736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


In [25]:
%%writefile ../src/FallbackTestBIN.json
{"contracts":{"../src/FallbackTest.sol:FallbackTest":{"bin":"608060405234801561001057600080fd5b50610190806100206000396000f3fe608060405234801561001057600080fd5b506004361061002f5760003560e01c8063e7f09e051461009a57610030565b5b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100a261011d565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100e25780820151818401526020810190506100c7565b50505050905090810190601f16801561010f5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040518060400160405280600b81526020017f646f696e672063616c6c4100000000000000000000000000000000000000000081525090509056fea2646970667358221220e82fa1bc4dc7ed37bbfad314b7799e14b2b537797a926ad6a5ca3317667e3f4064736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Writing ../src/FallbackTestBIN.json


In [27]:
%%writefile ../src/FallbackTestDeployFromFile.js
var Web3=require('web3');
var _abiJson = require('./FallbackTestABI.json');
var _binJson = require('./FallbackTestBIN.json');
//var fs=require('fs');
//var _str = fs.readFileSync("src/FallbackTestABI.json");
//var _json=JSON.parse(_str)

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName=Object.keys(_abiJson.contracts); // reading ['src//FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin=_binJson.contracts[contractName].bin;
//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/FallbackTestDeployFromFile.js


In [30]:
!node ../src/FallbackTestDeployFromFile.js

- contract name:  ../src/FallbackTest.sol:FallbackTest
Deploying the contract from 0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6
hash: 0xe02534f07c613103eefa8aadcc3aea5ab959bdfacf9bf56e35c7744a49c9868d
---> The contract deployed to: 0x0609642c3FC959Aec47968d990Ef448A17AD40Ca


In [58]:
%%writefile ../src/FallbackTestUseFromFile.js
var Web3=require('web3');
var _abiJson = require('./FallbackTestABI.json');

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    //web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhot:8345"));
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName=Object.keys(_abiJson.contracts); // reading ['..//src//FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    var _instance = new web3.eth.Contract(_abiArray, "0x0609642c3FC959Aec47968d990Ef448A17AD40Ca");
    var event = _instance.events.PrintLog(function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        }
    });

    _instance.methods.callA().call().then(console.log);  //null
    
    web3.eth.sendTransaction({from:accounts[0], to: "0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6"}); //fallback called
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}


doIt()

Overwriting ../src/FallbackTestUseFromFile.js


In [59]:
!node ../src/FallbackTestUseFromFile.js

- contract name:  ../src/FallbackTest.sol:FallbackTest
Account: 0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6
Balance before: 99999721286000000000
doing callA
Balance after: 99999721286000000000
Balance diff: 0


In [60]:
%%writefile ../src/MathMultiply7.sol
pragma solidity ^0.6.0;

contract Multiply7 {
   //event Print(uint);
   receive() external payable {}
   fallback() external payable {}
   function multiply(uint input) pure public returns (uint) {
      //emit Print(input * 7);
      return input * 7;
   }
   function getAddress() view public returns(address) {
       return address(this);
   }
}

contract Math {
    Multiply7 m7=new Multiply7();
    function deposit(uint amount) payable public {
        require(msg.value==amount);
    }
    function setM7(address payable _addr) public { m7 = Multiply7(_addr); }
    function multiply() view public returns(uint) {
        uint res=m7.multiply(8);
        //this.send11(); It does not send value.
        return res;
    }
    function send11M7() public payable {
        //m7.multiply.value(11)(9);
        //m7.multiply(9);
        address(m7).transfer(11);
    }
    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;
    }
    function getBalanceOfM7() public view returns(uint) {
        return address(m7).balance;
    }
    function getAddressOfM7() view public returns(address) {
        return address(m7);
   }    
}

Writing ../src/MathMultiply7.sol


In [62]:
!solc ../src/MathMultiply7.sol --combined-json abi

{"contracts":{"../src/MathMultiply7.sol:Math":{"abi":"[{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"deposit\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getAddressOfM7\",\"outputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfM7\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfThis\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"multiply\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"send11M7\",\"outputs\":[],\"stateMutability\":

In [63]:
%%writefile ../src/MathMultiply7ABI.json
{"contracts":{"../src/MathMultiply7.sol:Math":{"abi":"[{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"deposit\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getAddressOfM7\",\"outputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfM7\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfThis\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"multiply\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"send11M7\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address payable\",\"name\":\"_addr\",\"type\":\"address\"}],\"name\":\"setM7\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"}]"},"../src/MathMultiply7.sol:Multiply7":{"abi":"[{\"stateMutability\":\"payable\",\"type\":\"fallback\"},{\"inputs\":[],\"name\":\"getAddress\",\"outputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"input\",\"type\":\"uint256\"}],\"name\":\"multiply\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"pure\",\"type\":\"function\"},{\"stateMutability\":\"payable\",\"type\":\"receive\"}]"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Writing ../src/MathMultiply7ABI.json


In [64]:
!solc ../src/MathMultiply7.sol --combined-json bin

{"contracts":{"../src/MathMultiply7.sol:Math":{"bin":"60806040526040516100109061007e565b604051809103906000f08015801561002c573d6000803e3d6000fd5b506000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555034801561007857600080fd5b5061008b565b61013b8061048e83390190565b6103f48061009a6000396000f3fe6080604052600436106100705760003560e01c8063934d85b41161004e578063934d85b4146101015780639ef6a7041461012c578063b6b55f251461017d578063f3593cd0146101ab57610070565b806314eb4ad01461007557806356eff596146100cc57806360d2dab0146100f7575b600080fd5b34801561008157600080fd5b5061008a6101d6565b604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b3480156100d857600080fd5b506100e16101ff565b6040518082815260200191505060405180910390f35b6100ff61023f565b005b34801561010d57600080fd5b506101166102aa565b6040518082815260200191505060405180910390f35b34801561013857600080fd5b5061017b6004

In [65]:
%%writefile ../src/MathMultiply7BIN.json
{"contracts":{"../src/MathMultiply7.sol:Math":{"bin":"60806040526040516100109061007e565b604051809103906000f08015801561002c573d6000803e3d6000fd5b506000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555034801561007857600080fd5b5061008b565b61013b8061048e83390190565b6103f48061009a6000396000f3fe6080604052600436106100705760003560e01c8063934d85b41161004e578063934d85b4146101015780639ef6a7041461012c578063b6b55f251461017d578063f3593cd0146101ab57610070565b806314eb4ad01461007557806356eff596146100cc57806360d2dab0146100f7575b600080fd5b34801561008157600080fd5b5061008a6101d6565b604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b3480156100d857600080fd5b506100e16101ff565b6040518082815260200191505060405180910390f35b6100ff61023f565b005b34801561010d57600080fd5b506101166102aa565b6040518082815260200191505060405180910390f35b34801561013857600080fd5b5061017b6004803603602081101561014f57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506102b2565b005b6101a96004803603602081101561019357600080fd5b81019080803590602001909291905050506102f5565b005b3480156101b757600080fd5b506101c0610304565b6040518082815260200191505060405180910390f35b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16905090565b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc600b9081150290604051600060405180830381858888f193505050501580156102a7573d6000803e3d6000fd5b50565b600047905090565b806000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050565b80341461030157600080fd5b50565b6000806000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1663c6888fa160086040518263ffffffff1660e01b81526004018082815260200191505060206040518083038186803b15801561037a57600080fd5b505afa15801561038e573d6000803e3d6000fd5b505050506040513d60208110156103a457600080fd5b81019080805190602001909291905050509050809150509056fea2646970667358221220078c77129e12ffe7eb0d5bf4b5f83056a805c29a42fda17fa4ebe2352d49fb8564736f6c63430006040033608060405234801561001057600080fd5b5061011b806100206000396000f3fe60806040526004361060295760003560e01c806338cc4831146031578063c6888fa114608557602f565b36602f57005b005b348015603c57600080fd5b50604360d0565b604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b348015609057600080fd5b5060ba6004803603602081101560a557600080fd5b810190808035906020019092919050505060d8565b6040518082815260200191505060405180910390f35b600030905090565b600060078202905091905056fea2646970667358221220e28c8e8df9cbee18334bd3afd7efa1819ff89487fe360ff6cd7dac1c5a382f0164736f6c63430006040033"},"../src/MathMultiply7.sol:Multiply7":{"bin":"608060405234801561001057600080fd5b5061011b806100206000396000f3fe60806040526004361060295760003560e01c806338cc4831146031578063c6888fa114608557602f565b36602f57005b005b348015603c57600080fd5b50604360d0565b604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b348015609057600080fd5b5060ba6004803603602081101560a557600080fd5b810190808035906020019092919050505060d8565b6040518082815260200191505060405180910390f35b600030905090565b600060078202905091905056fea2646970667358221220e28c8e8df9cbee18334bd3afd7efa1819ff89487fe360ff6cd7dac1c5a382f0164736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Writing ../src/MathMultiply7BIN.json


In [66]:
%%writefile ../src/MathMultiply7DeployFromFile.js
var Web3=require('web3');
var _abiJson = require('./MathMultiply7ABI.json');
var _binJson = require('./MathMultiply7BIN.json');
//var fs=require('fs');
//var _str = fs.readFileSync("src/MathMultiply7ABI.json");
//var _json=JSON.parse(_str)

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractNames=Object.keys(_abiJson.contracts); //Math, Multiply7
contractName=contractNames[0]; // -> 'src/MathMultiply7.sol:Math', contractNames[1] -> Multiply7
console.log("- contract name: ", contractName); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin=_binJson.contracts[contractName].bin;
//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing ../src/MathMultiply7DeployFromFile.js


In [67]:
!node ../src/MathMultiply7DeployFromFile.js

- contract name:  ../src/MathMultiply7.sol:Math
Deploying the contract from 0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6
hash: 0x36b673d74069e61f41cdfb13d8d0272b5b4306fc14d609e1942da60b6ef859a3
---> The contract deployed to: 0x6Db404D94F1911600F89Bdc30e68A93c23366458


In [68]:
%%writefile ../src/MathMultiply7UseFromFile.js
var Web3=require('web3');
var _abiJson = require('./MathMultiply7ABI.json');
//var _binJson = require('./MathMultiply7BIN.json'); // not needed
//var fs=require('fs');
//var _str = fs.readFileSync("src/MathMultiply7ABI.json");
//var _json=JSON.parse(_str)

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    //web3 = new Web3(new Web3.providers.WebsocketProvider("ws://117.16.44.45:8345"));
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractNames=Object.keys(_abiJson.contracts); // reading ["src/MathMultiply7.sol:Math"]
contractName=contractNames[0];
console.log("- contract name: ", contractName); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
//_bin=_binJson.contracts[contractName].bin;
//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    var _instance = new web3.eth.Contract(_abiArray, "0x6Db404D94F1911600F89Bdc30e68A93c23366458");

    _instance.methods.multiply().call().then(console.log);
    _instance.methods.deposit(123).send({from:accounts[0], value:123});
    _instance.methods.getBalanceOfM7().call().then(console.log);
    await _instance.methods.send11M7().send({from:accounts[0]});
    _instance.methods.getBalanceOfM7().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Writing ../src/MathMultiply7UseFromFile.js


In [69]:
!node ../src/MathMultiply7UseFromFile.js

- contract name:  ../src/MathMultiply7.sol:Math
Account: 0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6
Balance before: 99991538426000000000
56
0
11
Balance after: 99990516465999999877
Balance diff: 1021959999995904


# 제어

In [ ]:
%%writefile src/Hello1.sol
pragma solidity ^0.6.0;
contract Hello1 {
    string private hello;
    address private owner;
    event PrintLog(address addr, string s);
    constructor(string memory _hello) public {
        hello = _hello;
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    modifier isOwner() {
        if (msg.sender != owner) {
            revert();
        }
        _; //continue executing rest of method body
    }
    function setHello() public {
        string memory s = "";
        if (msg.sender == owner) {
            s = "Hello";
            emit PrintLog(msg.sender, s);
        } else {
            s = "Olleh";
            emit PrintLog(msg.sender, s);
        }
        hello = s;
    }
    function compareTo(string memory _str) view public returns(bool) {
        bool isEqual = false;
        //if (hello == _str) {
        if (keccak256(bytes(hello)) == keccak256(bytes(_str))) {
            isEqual = true;
        }
        return isEqual;
    }
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}